In [1]:
'''
This is to scrape att.com's iphone reviews.
There are 1117 reviews and ratings for iphone X.

written by Soi Kang
3/30/2019
'''

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
driver.get("https://www.att.com/buy/phones/apple-iphone-x-64gb-space-gray.html")

# need to navigate through reviews.
# problem for this website is that the url doesn't change no matter what I navigate through
# but selenium can do it

#this is to get to the reviews from the original url
review_button_path = '//*[@id="accordion-control-reviews"]/span[1]'
wait = WebDriverWait(driver, 5)
review_button = wait.until(EC.visibility_of_element_located((By.XPATH, review_button_path)))
driver.execute_script("arguments[0].click();", review_button)

In [2]:
# this below retreives all the text in the review container
# but don't need it....
#review_container = driver.find_element_by_id("BVRRContainer")

# get the reviews and rating for the first page
#reviews = []
#ratings = []
review_rating = []

review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
for el in review_list:
    #print(el.find_element_by_class_name("bv-content-summary-body-text").text)
    #print(el.find_element_by_class_name("bv-rating-ratio").text)
    review_rating.append(el.find_element_by_class_name("bv-content-summary-body-text").text)
    review_rating.append(el.find_element_by_class_name("bv-rating-ratio").text)

In [3]:
from selenium.common import exceptions
import time

# get the reviews starting from the 2nd page
# until there is no button to click
# there are about 37ish pages total since there are about 1100 reviews
# (probably) 30 pages per reviews except for the first page
i=2
reviews_ratings=[]
#wait = WebDriverWait(driver, 30)

while(True):
    try :
        next_review_path = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/ul/li[2]/button' 
        review_next_button = WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, next_review_path)))
        driver.execute_script("arguments[0].click();", review_next_button)
        time.sleep(3)
  
        # append the reviews and ratings to a list
        review_list = driver.find_elements_by_css_selector('.bv-content-item.bv-content-top-review.bv-content-review')
        for el in review_list:
            review_rating.append(el.find_element_by_class_name("bv-content-summary-body-text").text)
            review_rating.append(el.find_element_by_class_name("bv-rating-ratio").text)
        time.sleep(3)
        print(i)
        i=i+1
    
    #except exceptions.StaleElementReferenceException:
    except exceptions.StaleElementReferenceException as e:
        raise e
    
    except exceptions.TimeoutException :
        break
    

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [16]:
# this should have 1117*2=2234 elements
# since I have added both reviews and ratings in one list
# this was done to compare the reviews and ratings together
print(len(review_rating))
print(1117*2)

# separate the list to two
# one for review and the other for 
ratings = []
reviews = []

# even number to be reviews (starting at index 0)
# review number to be ratings

for i in range(len(review_rating)) :
    if((i%2)==0):
        reviews.append(review_rating[i])
    
    else :
        ratings.append(review_rating[i])
    

#now we have to have 1117 elements in each separated list
print(len(reviews))
print(len(ratings))    
    

2234
2234
1117
1117


In [34]:
#print(reviews[0:10])
#print(ratings[0:10])

#now, let's get just the numbers in the ratings list and get rid of all the unnecessary strings
#if you see the first instance of a number, that's the rating we should get
import re

#print(ratings[0])
#print(re.search(r'\d+', ratings[0]).group())

numbers = []
for rating in ratings :
    
    numbers.append(re.search(r'\d+', rating).group())

print(ratings[-20:])
print(numbers[-20:])
print(len(numbers))

['★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n3 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n5 out of 5 stars.', '★★★★★\n★★★★★\n3 out of 5 stars.']
['5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5', '3', '5', '5', '5', '5', '5', '5', '3']
1117


In [38]:
# combine the two lists as a dataframe
import pandas as pd

df = pd.DataFrame({"review" : reviews, "rating" : numbers})
df['source'] = "att.com"
df['phone'] = "iphone"
    
    
#reorder columns
cols = ['source','phone','review','rating']
final = df.reindex(columns=cols)        

In [40]:
len(final)
final.to_csv('att_iphone.csv', index=False)